In [6]:
import os

### Определяем геномы, среди которых будем искать пересечения

In [40]:
genome_1 = "/media/eternus1/projects/zilov/data/mycoplasma/genomes/GCF_000008365.1_ASM836v1_genomic.fna"
genome_2 = "/media/eternus1/projects/zilov/data/mycoplasma/genomes/GCF_000011225.1_ASM1122v1_genomic.fna"
genome_3 = "/media/eternus1/projects/zilov/data/mycoplasma/genomes/GCF_000012765.1_ASM1276v1_genomic.fna"

genomes = [genome_1, genome_2, genome_3]

### Функции которые нам пригодятся

In [25]:
def fasta_read(path_to_fasta_file):
    "Returns dicttionary with header:sequence"
    fasta = {}
    header = None
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    fasta[header] = "".join(seq)
                header = line[1:].split()[0]
                seq = []
            else:
                seq.append(line)
        if header:
            fasta[header] = "".join(seq)
    return fasta


def seq_kmers(k, sequence):
    """Extracts all k-mers from genome

    Args:
        k (int): Length of k-mers
        sequence (str): nucleotide/aminoacid sequence
    
    Returns:
        list: all k-mers from genome
    """
    
    kmers = []
    
    for s in range(0, len(sequence) - k + 1):
        kmer = sequence[s:s+k]
        kmers.append(kmer)
    return kmers


def reverse_complement(sequence):
    """
    Args:
        seq (str): nucleotide sequence

    Returns:
        Reverse complement string
    """
    complementary_table = str.maketrans({'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'})
    revcomp_seq = sequence[::-1].translate(complementary_table)

    return revcomp_seq

### Выделяем все уникальные кмеры из всех геномов

In [26]:
all_unique_kmers = {}

for i, genome in enumerate(genomes):
    
    genome_name = os.path.splitext(os.path.basename(genome))[0]
    fasta_dict = fasta_read(genome)
    
    for k in range(20, 31, 2):
        genome_kmers_set = set()
        kmers_key = f"{k}:{genome_name}"
        for header, seq in fasta_dict.items():
            unique_kmers_forward = set(seq_kmers(k, seq))
            unique_kmers_reverse = set(seq_kmers(k, reverse_complement(seq)))
            genome_kmers_set.update(unique_kmers_forward, unique_kmers_reverse)
        all_unique_kmers[kmers_key] = genome_kmers_set

### Простой пример поиска пересечения

In [47]:
genome_1_kmers = all_unique_kmers["30:GCF_000008365.1_ASM836v1_genomic"]
genome_2_kmers = all_unique_kmers["30:GCF_000011225.1_ASM1122v1_genomic"]
genome_3_kmers = all_unique_kmers["30:GCF_000012765.1_ASM1276v1_genomic"]

common_kmers = genome_1_kmers.intersection(genome_2_kmers, genome_3_kmers)
print(common_kmers)

{'TGTATTACCGCGGCTGCTGGCACATAGTTA', 'GTATTACCGCGGCTGCTGGCACATAGTTAG', 'GGCGGCCGTAACTATAACGGTCCTAAGGTA', 'CCTACTGATTACAAGTCAGTTGCTCTGCCT', 'ATGTATTACCGCGGCTGCTGGCACATAGTT', 'GTAACTATAACGGTCCTAAGGTAGCGAAAT', 'CTATGTATTACCGCGGCTGCTGGCACATAG', 'CCGTAACTATAACGGTCCTAAGGTAGCGAA', 'CGGCGGCCGTAACTATAACGGTCCTAAGGT', 'GTGAACGGCGGCCGTAACTATAACGGTCCT', 'GCTACCTTAGGACCGTTATAGTTACGGCCG', 'GAACGGCGGCCGTAACTATAACGGTCCTAA', 'TAGGACCGTTATAGTTACGGCCGCCGTTCA', 'CTACCTTAGGACCGTTATAGTTACGGCCGC', 'TTACCGCGGCTGCTGGCACATAGTTAGCCG', 'TACCTTAGGACCGTTATAGTTACGGCCGCC', 'TTTCGCTACCTTAGGACCGTTATAGTTACG', 'CTAACTATGTGCCAGCAGCCGCGGTAATAC', 'CTTAGGACCGTTATAGTTACGGCCGCCGTT', 'TAACTATGTGCCAGCAGCCGCGGTAATACA', 'CCTATGTATTACCGCGGCTGCTGGCACATA', 'TATGTATTACCGCGGCTGCTGGCACATAGT', 'ATTACCGCGGCTGCTGGCACATAGTTAGCC', 'GCGGCCGTAACTATAACGGTCCTAAGGTAG', 'TTCGCTACCTTAGGACCGTTATAGTTACGG', 'ACCTACTGATTACAAGTCAGTTGCTCTGCC', 'GGTGAACGGCGGCCGTAACTATAACGGTCC', 'CGCTACCTTAGGACCGTTATAGTTACGGCC', 'GGCCGTAACTATAACGGTCCTAAGGTAGCG', 'GGCAGAGCAACT

### Если лень вводить для каждого генома вручную

In [48]:
intersec = {}

for k in range(30, 19, -2):
    k_intersection = None
    k_keys = [] # записываем все ключи с одинаковым K
    print(k)
    for key in all_unique_kmers.keys():
        if int(key.split(":")[0]) == k:
            k_keys.append(key)
    print(k_keys)
    
    # пробегаемся по всем множествам 
    for i in range(len(k_keys) - 1):
        
        if k_intersection:
            last_intersection = k_intersection
            set_1 = k_intersection    
        
        else:
            if i == 0:
                set_1 = all_unique_kmers[k_keys[0]]
            elif last_intersection:
                intersec[f"not_full:{i}/{len(k_keys)}:{k}"] = last_intersection
                k_intersection = None
                print("miss")
                break
            else:
                print(f"No intersections with k = {k}")
        
        set_2 = all_unique_kmers[k_keys[i+1]]
        
        
        print(f"len of set_1 = {len(set_1)}, set_2 = {len(set_2)}")
        
        k_intersection = set_1.intersection(set_2)
        
        if i == len(k_keys) - 2:
            if k_intersection:
                intersec[k] = k_intersection
                print("intersection length:", len(k_intersection))
                k_intersection = None
                print("nice")
            else:
                print("no intersections with last set")
        else:
            intersec[k] = last_intersection
            print("add last")

30
['30:GCF_000008365.1_ASM836v1_genomic', '30:GCF_000011225.1_ASM1122v1_genomic', '30:GCF_000012765.1_ASM1276v1_genomic']
len of set_1 = 1523201, set_2 = 2639408
add last
len of set_1 = 134, set_2 = 1984923
intersection length: 52
nice
28
['28:GCF_000008365.1_ASM836v1_genomic', '28:GCF_000011225.1_ASM1122v1_genomic', '28:GCF_000012765.1_ASM1276v1_genomic']
len of set_1 = 1522186, set_2 = 2636473
add last
len of set_1 = 198, set_2 = 1983289
intersection length: 66
nice
26
['26:GCF_000008365.1_ASM836v1_genomic', '26:GCF_000011225.1_ASM1122v1_genomic', '26:GCF_000012765.1_ASM1276v1_genomic']
len of set_1 = 1521099, set_2 = 2633031
add last
len of set_1 = 296, set_2 = 1981448
intersection length: 88
nice
24
['24:GCF_000008365.1_ASM836v1_genomic', '24:GCF_000011225.1_ASM1122v1_genomic', '24:GCF_000012765.1_ASM1276v1_genomic']
len of set_1 = 1519916, set_2 = 2628979
add last
len of set_1 = 446, set_2 = 1979315
intersection length: 124
nice
22
['22:GCF_000008365.1_ASM836v1_genomic', '22:GCF_

### Домашнее задание

1) Скачать три генома: референсные E.coli, P. aueroginosa, B. bifidum

2) Найти нуклеотидные и аминокислотные пересечения между ними, определить максимальную длину

3) Скачать 10 геномов разных штаммов E.coli, найти пересечения по всем геномам

In [107]:
from collections import defaultdict

def mmseq_clusters_parser(mmseq_cluster_tsv):
    clusters = defaultdict(list)
    with open(mmseq_cluster_tsv) as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            line = line.strip().split()
            cluster_name = line[0]
            cluster_part = line[1] 
            if cluster_name == cluster_part:
                previous_cluster = cluster_name
                clusters[previous_cluster].append(cluster_part)
            else:
                clusters[previous_cluster].append(cluster_part)
    return clusters

def mmseq_clusters_numbers(clusters_dict):
    clusters_number = []
    for cluster, parts in clusters_dict.items():
        clusters_number.append([cluster, len(parts)])
    clusters_number.sort(key = lambda x: x[1], reverse = True)
    return clusters_number

def fasta_read(path_to_fasta_file):
    "Returns dictionary header:sequence"
    fasta = {}
    header = None
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    fasta[header] = "".join(seq)
                header = line[1:].split()[0]
                seq = []
            else:
                seq.append(line)
        if header:
            fasta[header] = "".join(seq)
    return fasta

In [104]:
myco_nucl_clusters = mmseq_clusters_parser("/media/eternus1/projects/zilov/data/mycoplasma/mmseq_aminoprots/mycoprot_cluster.tsv")
myco_nucl_clusters_numbers = mmseq_clusters_numbers(myco_nucl_clusters) 

In [108]:
from glob import glob

amino_fasta = glob("/home/dzilov/data_zilov/data/mycoplasma/proteins_amino/*.faa")

cluster_fasta_dict = {}

for fasta in amino_fasta:
    genome = fasta_read(fasta)
    for prot_id in myco_nucl_clusters["WP_036444774.1"]:
        if prot_id in genome.keys():
            cluster_fasta_dict[prot_id] = genome[prot_id]

In [110]:
fasta_cluster = "/home/dzilov/data_zilov/data/mycoplasma/clusters_fasta_aa/WP_036444774.1.fasta"

with open(fasta_cluster, "w") as fw:
    for header, seq in cluster_fasta_dict.items():
        fw.write(f">{header}\n{seq}\n")

### И этот fasta cluster прогоняем muscle

После визуализируем

### ДЗ

1) Качаем белки по микоплазме

2) Кластеризуем

3) Проверяем что белки в кластере относятся к разным бактериям (чтобы не было многокопийных)

4) Выравниваем

5) Визуализируем (сами найдите тул)